# Movie Recommendation System using KNN
**Dataset:** MovieLens 1M  
**Method:** Item-Based Collaborative Filtering  
**Metric:** Cosine Similarity


In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv("ml-1m/ratings.dat",delimiter="::", names = ["user_id", "movie_id", "rating", "time_stamp"])

## User–Movie Rating Matrix

We transform the ratings data into a movie × user matrix.
- Rows represent movies
- Columns represent users
- Missing ratings are filled with 0


In [ ]:
df = df.pivot(index="movie_id", columns="user_id", values = "rating").fillna(0)

In [ ]:
df_sparse = csr_matrix(df.values)

## KNN Model

We use an item-based KNN model with cosine similarity.
Cosine similarity works well for sparse high-dimensional data,
which is typical in recommender systems.


In [ ]:
knn = NearestNeighbors(metric="cosine", algorithm="brute")
knn.fit(df_sparse)

In [ ]:
movies = pd.read_csv("ml-1m/movies.dat",delimiter="::", names = ["movie_id", "movie_name", "genre"])

In [ ]:
name_to_id = dict(
    zip(movies["movie_name"].str.lower(), movies["movie_id"])
)

## Recommendation Strategy

Given a movie:
1. Find its vector in the rating matrix
2. Compute cosine distance to other movies
3. Select nearest neighbors
4. Attach movie metadata for readability


In [ ]:
def suggest(name:str):
    """The movie name should include the year of release in parentheses in the end"""
    name = name.lower().strip()
    if name not in name_to_id:
        return "Movie not found"
    else:
        id = name_to_id[name]
    
    idx = df.index.get_loc(id)
    distances, indices = knn.kneighbors(
        df_sparse[idx],
        n_neighbors= 6
    )

    similar_movie_ids = df.index[indices.flatten()]
    similar_distances = distances.flatten()

    results = pd.DataFrame({
        "movie_id": similar_movie_ids,
        "distance": similar_distances
    })

    results = results.merge(movies, on="movie_id")
    results = results[results["distance"] > 0]
    results = results.sort_values("distance")

    print(results[["movie_name", "genre", "distance"]])

In [ ]:
suggest("pulp fiction (1994)")

## Conclusion

This project demonstrates an item-based collaborative filtering
system built using KNN and cosine similarity. The model produces
interpretable and meaningful recommendations.
